<a href="https://colab.research.google.com/github/nomi181472/MS-DS-Practice/blob/main/cnn/simplecnn/pytorch_convolutional.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

In [2]:
train_dataset=torchvision.datasets.FashionMNIST(
    root=",",
    train=True,
    transform=transforms.ToTensor(),
    download=True
)

100%|██████████| 26421880/26421880 [00:01<00:00, 13610658.40it/s]


Extracting ,/FashionMNIST/raw/train-images-idx3-ubyte.gz to ,/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 229052.12it/s]


Extracting ,/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ,/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:01<00:00, 4267879.31it/s]


Extracting ,/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ,/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 23469866.30it/s]

Extracting ,/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ,/FashionMNIST/raw



In [9]:
  print(train_dataset.data.shape)
  print(train_dataset.targets.shape)

torch.Size([60000, 28, 28])
torch.Size([60000])


In [11]:
test_dataset=torchvision.datasets.FashionMNIST(
    root=",",
    train=False,
    transform=transforms.ToTensor(),
    download=True
)

In [12]:
  print(test_dataset.data.shape)
  print(test_dataset.targets.shape)

torch.Size([10000, 28, 28])
torch.Size([10000])


In [75]:
#define the model

class CNN(nn.Module):
  def __init__(self,K):
    super(CNN,self,).__init__()
    self.conv_layers=nn.Sequential(
        nn.Conv2d(in_channels=1,out_channels=32,kernel_size=3,stride=2),
        nn.ReLU(),
        nn.Conv2d(in_channels=32,out_channels=64,kernel_size=3,stride=2),
        nn.ReLU(),
        nn.Conv2d(in_channels=64,out_channels=128,kernel_size=3,stride=2),
        nn.ReLU()

    )
    self.dense_layers=nn.Sequential(
        nn.Dropout(0.2),
        nn.Linear(128*2*2,512),
        nn.ReLU(),
        nn.Dropout(0.2),
        nn.Linear(512,K)
    )
  def forward(self,x):

    out=self.conv_layers(x)
    out=out.view(out.size(0),-1)

    out=self.dense_layers(out)

    return out

In [76]:
model=CNN(10)
device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
model.to(device)

cpu


CNN(
  (conv_layers): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(2, 2))
    (1): ReLU()
    (2): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2))
    (3): ReLU()
    (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2))
    (5): ReLU()
  )
  (dense_layers): Sequential(
    (0): Dropout(p=0.2, inplace=False)
    (1): Linear(in_features=512, out_features=512, bias=True)
    (2): ReLU()
    (3): Dropout(p=0.2, inplace=False)
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)

In [77]:
criterion=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model.parameters())

In [78]:
batch_size=128
train_loader=torch.utils.data.DataLoader(
    dataset=train_dataset,
    batch_size=batch_size,
    shuffle=True
)
test_loader=torch.utils.data.DataLoader(
    dataset=test_dataset,
    batch_size=batch_size,
    shuffle=True
)

In [82]:
def batch_gd(model,criterion,optimizer,train_loader,test_loader,epochs):
  train_losses=np.zeros(epochs)
  test_losses=np.zeros(epochs)
  for it in range(epochs):
    model.train()
    t0=datetime.now()
    train_loss=[]


    for inputs,targets in train_loader:
      inputs=inputs.to(device)
      targets=targets.to(device)
      optimizer.zero_grad()
      outputs=model(inputs)
      optimizer.zero_grad()
      loss=criterion(outputs,targets)
      loss.backward()
      optimizer.step()
      train_loss.append(loss.item())

    train_loss=np.mean(train_loss)
    model.eval()
    test_loss=[]
    for inputs, targets in test_loader:
      inputs,targets =inputs.to(device),targets.to(device)
      outputs=model(inputs)
      loss=criterion(outputs,targets)
      test_loss.append(loss.item())
    test_loss=np.mean(test_loss)

    train_losses[it]=train_loss
    test_losses[it]=test_loss

    t1=datetime.now()-t0
    print(f"epoch {it+1}/{epochs} train loss: {train_loss} test loss {test_loss}")
  return train_losses, test_losses


train_losses,test_losses=batch_gd(model,criterion,optimizer,train_loader,test_loader, epochs=15)









epoch 1/15 train loss: 0.3790844683962336 test loss 0.3659041391897805
epoch 2/15 train loss: 0.3480894373678195 test loss 0.35368625819683075
epoch 3/15 train loss: 0.3203040600649075 test loss 0.33703524798532075
epoch 4/15 train loss: 0.29925402506455173 test loss 0.3154578791766227
epoch 5/15 train loss: 0.2789844416860324 test loss 0.3111358801775341
epoch 6/15 train loss: 0.26184178567898553 test loss 0.3086193449135068
epoch 7/15 train loss: 0.24689339018706827 test loss 0.28973068768465066
epoch 8/15 train loss: 0.23183438418580016 test loss 0.2999078755891776
epoch 9/15 train loss: 0.2202878930707222 test loss 0.28769916352591934
epoch 10/15 train loss: 0.20818318356710203 test loss 0.2909632685441005
epoch 11/15 train loss: 0.1961625371851138 test loss 0.2829446083382715
epoch 12/15 train loss: 0.18380327093829987 test loss 0.29264108534855177
epoch 13/15 train loss: 0.17457176776710095 test loss 0.29367198667760136
epoch 14/15 train loss: 0.16536932869919582 test loss 0.3015